In [167]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [168]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
from embedding import CustomEmbedding
from transformers import EncoderDecoderTransformer
from utils import padding_collate_fn

from generate_data import RandomIntegerDataset

In [169]:
n_real_tokens = 10
PAD_TOKEN_IDX = n_real_tokens
SOS_TOKEN_IDX = n_real_tokens + 1
EOS_TOKEN_IDX = n_real_tokens + 2
vocab_size = n_real_tokens + 3
D_MODEL = 16

embeddings = CustomEmbedding(vocab_size = vocab_size + 3, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[-1.1675, -2.1670, -0.5001, -0.5215,  0.0514, -0.6643,  1.6151, -0.8825,
          0.8223, -1.1218,  1.4389, -0.9955, -2.9037, -0.9089, -0.8950, -1.5548],
        [-0.6461,  0.9344,  0.6238,  0.8738,  0.3457, -0.4177, -1.3645,  0.2604,
          1.5974,  0.0135, -0.6102,  0.0634, -2.0247, -0.1224, -0.5331,  0.3599]],
       grad_fn=<EmbeddingBackward0>)


In [170]:
MAX_CONTEXT_WINDOW = 50

BATCH_SIZE = 2
MIN_SEQ_LEN = 2
MAX_SEQ_LEN = min(10, MAX_CONTEXT_WINDOW)

rand_ds = RandomIntegerDataset(2, 10, 10, [i for i in range(10)])
dataloader = DataLoader(rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [171]:
input, label = next(iter(dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[ 0,  4,  4,  5,  8,  4,  4],
        [ 6,  4,  3,  9, 10, 10, 10]])
tensor([[11,  0,  4,  4,  4,  4,  5,  8],
        [11,  3,  4,  6,  9, 10, 10, 10]])
tensor([[ 0,  4,  4,  4,  4,  5,  8, 12],
        [ 3,  4,  6,  9, 12, 10, 10, 10]])


In [172]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX, reduction = 'sum')

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 1, 
                    num_decoder_layers = 1, 
                    dim_feedforward = 32, 
                    max_context_window = MAX_CONTEXT_WINDOW)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-3, momentum = 0.9, weight_decay = 1e-4)

In [173]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer):
    
    model.train()

    epoch_loss = 0.0

    for i, ((source, target), label) in enumerate(dataloader):
        print(f'Processing Batch {i+1}...')

        # FORWARD
        pred = model(source, target)
        batch_loss = loss_fn(pred, label)

        # LOG
        epoch_loss += batch_loss.item()

        # BACKWARD
        batch_loss.backward()

        # OPTIMIZE
        optimizer.step()
        optimizer.zero_grad()

    average_epoch_loss = epoch_loss / len(dataloader.dataset)

    return average_epoch_loss

def run_test_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module):
    
    model.eval()

In [174]:
EPOCHS = 1

training_losses = list()

for i in range(EPOCHS):

    training_loss = run_train_epoch(dataloader, model, loss_fn, optim)
    training_losses.append(training_loss)

Processing Batch 1...


RuntimeError: The size of tensor a (2) must match the size of tensor b (4) at non-singleton dimension 1